### Graphs Representation 

#### Properties
Adjacency List: for each vertex u, store a list of its neighbors. $\newline$
Adjacency Matrix: an n×n boolean array M, where $M[u][v]=1 iff (u,v)∈E$.


### Building the adjacencies components:

Pseudocode:
INPUT: edge list E, number of vertices n
1. // Adjacency List
   for u in 0…n−1:
     Adj[u] ← empty list
   for each (u,v) in E:
     Adj[u].append(v)
     Adj[v].append(u)          // if undirected

2. // Adjacency Matrix
   for u in 0…n−1:
     for v in 0…n−1:
       M[u][v] ← 0
   for each (u,v) in E:
     M[u][v] ← 1
     M[v][u] ← 1               // if undirected


In [ ]:
def build_graph(edges, n):
    # edges: list of (u,v), vertices 0…n−1
    Adj = [[] for _ in range(n)]
    M   = [[0]*n for _ in range(n)]
    for u,v in edges:
        Adj[u].append(v)
        Adj[v].append(u)
        M[u][v] = 1
        M[v][u] = 1
    return Adj, M


### Depth-First Search
(traverses all reachable vertices, records discovery/finish for each)

#### Recurisive DFS

Pseudocode:
GLOBAL time ← 0
for u in 0…n−1:
  color[u] ← WHITE
DFS():
  for u in 0…n−1:
    if color[u]==WHITE:
      DFS-Visit(u)

FUNCTION DFS-Visit(u):
  color[u] ← GRAY
  discovery[u] ← ++time
  for v in sorted(Adj[u]):
    if color[v]==WHITE:
      parent[v] ← u
      DFS-Visit(v)
  color[u] ← BLACK
  finish[u] ← ++time


In [ ]:
def dfs(Adj):
    n = len(Adj)
    color = ['W']*n
    disc = [0]*n
    fin  = [0]*n
    parent = [None]*n
    time = 0

    def visit(u):
        nonlocal time
        color[u] = 'G'
        time += 1; disc[u] = time
        for v in sorted(Adj[u]):
            if color[v]=='W':
                parent[v] = u
                visit(v)
        color[u] = 'B'
        time += 1; fin[u] = time

    for u in range(n):
        if color[u]=='W':
            visit(u)
    return parent, disc, fin


#### Iterative DFS

Pseudocode:
FUNCTION DFS-Iterative(s):
  for u in 0…n−1: visited[u]=false
  stack ← empty
  PUSH(stack, s)
  while stack not empty:
    u ← POP(stack)
    if not visited[u]:
      visited[u]=true
      for v in sorted(Adj[u], descending):
        if not visited[v]:
          parent[v]=u
          PUSH(stack, v)


In [ ]:
def dfs_iterative(Adj, s=0):
    n = len(Adj)
    visited = [False]*n
    parent = [None]*n
    stack = [s]
    while stack:
        u = stack.pop()
        if not visited[u]:
            visited[u] = True
            for v in sorted(Adj[u], reverse=True):
                if not visited[v]:
                    parent[v] = u
                    stack.append(v)
    return parent


### Breadth-First Search 
(used for shortest path tree in unweighted graph)

Pseudocode:
FUNCTION BFS(s):
  for u in 0…n−1:
    dist[u] = ∞; parent[u]=null
  dist[s] = 0
  Q ← empty queue
  ENQUEUE(Q, s)
  while Q not empty:
    u ← DEQUEUE(Q)
    for v in sorted(Adj[u]):
      if dist[v]==∞:
        dist[v] = dist[u]+1
        parent[v] = u
        ENQUEUE(Q, v)


In [ ]:
from collections import deque

def bfs(Adj, s=0):
    n = len(Adj)
    dist = [None]*n
    parent = [None]*n
    dist[s] = 0
    Q = deque([s])
    while Q:
        u = Q.popleft()
        for v in sorted(Adj[u]):
            if dist[v] is None:
                dist[v] = dist[u] + 1
                parent[v] = u
                Q.append(v)
    return parent, dist


### Cycle Detection in Undirected Graph

Tests if any cycle exists; DFS, but ignore edge back to parent 

Pseudoceode:
for u in 0…n−1: visited[u]=false
FUNCTION DFS-Cycle(u, p):
  visited[u]=true
  for v in Adj[u]:
    if not visited[v]:
      if DFS-Cycle(v, u): return true
    else if v ≠ p:
      return true
  return false

for u in 0…n−1:
  if not visited[u] and DFS-Cycle(u, -1):
    return true
return false


In [ ]:
def has_cycle(Adj):
    n = len(Adj)
    visited = [False]*n
    def dfs(u, p):
        visited[u] = True
        for v in Adj[u]:
            if not visited[v]:
                if dfs(v, u):
                    return True
            elif v != p:
                return True
        return False

    for u in range(n):
        if not visited[u]:
            if dfs(u, -1):
                return True
    return False


### Number of Shortest Paths between s and t
Counts how many distinct shortest length paths; augment BFS

Pseudocode:
for u: dist[u]=∞; count[u]=0
dist[s]=0; count[s]=1
Q ← [s]
while Q:
  u ← DEQUEUE(Q)
  for v in Adj[u]:
    if dist[v]==∞:
      dist[v]=dist[u]+1
      count[v]=count[u]
      ENQUEUE(Q,v)
    else if dist[v]==dist[u]+1:
      count[v] += count[u]
return count[t]


In [ ]:
from collections import deque

def num_shortest_paths(Adj, s, t):
    n = len(Adj)
    dist = [None]*n
    count = [0]*n
    dist[s] = 0; count[s] = 1
    Q = deque([s])
    while Q:
        u = Q.popleft()
        for v in Adj[u]:
            if dist[v] is None:
                dist[v] = dist[u] + 1
                count[v] = count[u]
                Q.append(v)
            elif dist[v] == dist[u] + 1:
                count[v] += count[u]
    return count[t]
